# MgO_example.ipynb
### Bobby Appleton (edited from qe_test.ipynb by Kat Nykiel)
## Objectives
- query MgO structure from materials project
- strain structure using pymatgen
- generate QE input files with pymatgen for ionic relax
- generate QE input files with pymatgen for volume relax with external pressure
- submit QE jobs from notebook
- plot and fit EOS for MgO

### Query structure from materials project (new API)

In [1]:
# Read in new API key
import os, stat
from IPython.display import clear_output

# Read in new Materials Project API key, if one exists
with open(os.path.expanduser('~/.mpkey.txt'), 'r') as f:
    try:
        key = f.readlines()[0]
    except:
        key = ""
        
# Check if API key already exists, skip try-except
if not key:
    # Prompt user for API key
    try:
        user = str(input())
        clear_output()
        if not user.isalnum():
            raise TypeError('Wrong Key')
        if user == None:
            raise TypeError('Empty')
        with open(os.path.expanduser('~/.mpkey.txt'), 'w') as keyfile:
            keyfile.write(user)
        os.chmod(os.path.expanduser('~/.mpkey.txt'), stat.S_IREAD | stat.S_IWRITE)
        del user

        print("Success")
    except:
        print("Something seems wrong with your key")

In [2]:
from mp_api.client import MPRester

# Query using new API for material with ID of mp-1265 (MgO)
with MPRester(key) as m:
    data = m.summary.search(material_ids=["mp-1265"])

/apps/share64/debian10/anaconda/anaconda-7/envs/MATERIALS/lib/python3.8/site-packages/mp_api/client/client.py:139: builtins.UserWarning: Problem loading MPContribs client: 
/apps/share64/debian10/anaconda/anaconda-7/envs/MATERIALS/lib/python3.8/site-packages/maggma/api/utils.py:107: builtins.RuntimeWarning: fields may not start with an underscore, ignoring "_framework_formula"
/apps/share64/debian10/anaconda/anaconda-7/envs/MATERIALS/lib/python3.8/site-packages/maggma/api/utils.py:107: builtins.RuntimeWarning: fields may not start with an underscore, ignoring "_stable_entries"
/apps/share64/debian10/anaconda/anaconda-7/envs/MATERIALS/lib/python3.8/site-packages/maggma/api/utils.py:107: builtins.RuntimeWarning: fields may not start with an underscore, ignoring "_unstable_entries"


Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
struct = data[0].structure
print(struct)

Full Formula (Mg1 O1)
Reduced Formula: MgO
abc   :   3.009789   3.009789   3.009789
angles:  60.000000  60.000000  60.000000
Sites (2)
  #  SP      a    b    c    magmom
---  ----  ---  ---  ---  --------
  0  Mg    0    0    0          -0
  1  O     0.5  0.5  0.5        -0


### Strain structure using pymatgen

In [4]:
# Check lattice parameters before strain
struct.lattice.abc

(3.0097887004120407, 3.0097887004120407, 3.0097887004120407)

In [5]:
# Apply strain to lattice
struct.apply_strain(-0.1)

In [6]:
# Check new lattice vectors
struct.lattice.abc

(2.7088098303708366, 2.7088098303708366, 2.7088098303708366)

### Generate QE input files for ionic relax using pymatgen

In [7]:
from pymatgen.io.pwscf import PWInput

# Define input set
input_set = PWInput(struct, 
                    pseudo={'Mg':'Mg.PBE.UPF','O':'O.PBE.UPF'}, 
                    control={'pseudo_dir':'./',
                             'calculation':'relax',
                             'outdir':'./',
                             'tstress':True},
                    system={'ecutwfc':40},
                    kpoints_grid=[12,12,12])

input_set.write_file(filename='ion_relax.in')

# Define input and output files
ion_rlx_input_file = open('ion_relax.in','a')
ion_rlx_input_file.close()

ion_rlx_output_file = open("ion_relax.out", "w")

In [8]:
# Run simulation
pseudos = "-i ./pseudo/Mg.PBE.UPF ./pseudo/O.PBE.UPF"
COMMANDrlx = f"espresso-6.8_pw > {ion_rlx_output_file.name}"
!submit -n 1 -w '01:00:00' -e QE_DISABLE_GGA_PBE=0 --runName irlx {COMMANDrlx} {pseudos} -i {ion_rlx_input_file.name}
ion_rlx_output_file.close()

### Extract QE outputs using pymatgen

In [9]:
from pymatgen.io.pwscf import PWOutput

# Extract outputs
output_set = PWOutput('irlx.stdout')

In [10]:
output_set.final_energy

-75.45258451

In [11]:
[print(k) for k,v in output_set.data.items()]

lattice_type
ecut
celldm1
celldm2
celldm3
celldm4
celldm5
celldm6
nkpts
energies


[None, None, None, None, None, None, None, None, None, None]

In [12]:
# Extract stress manually
output = open('irlx.stdout', "r")
lines = output.readlines()
P = []
job_time = []
for line in lines:
    if 'P=' in line:
        data = line.split()
        P.append(float(data[5]))
print(P)

[872.4]


### Generate QE input files for volume relax using pymatgen

In [13]:
from pymatgen.io.pwscf import PWInput

# Define input set
input_set = PWInput(struct, 
                    pseudo={'Mg':'Mg.PBE.UPF','O':'O.PBE.UPF'}, 
                    control={'pseudo_dir':'./',
                             'calculation':'vc-relax',
                             'outdir':'./',
                             'tstress':True},
                    system={'ecutwfc':40},
                    cell={'press':P[0]},
                   kpoints_grid=[12,12,12])

input_set.write_file(filename='vol_relax.in')

# Define input and output files
vol_rlx_input_file = open('vol_relax.in','a')
vol_rlx_input_file.close()

vol_rlx_output_file = open("vol_relax.out", "w")

In [14]:
# Run simulation
pseudos = "-i ./pseudo/Mg.PBE.UPF ./pseudo/O.PBE.UPF"
COMMANDvcrlx = f"espresso-6.8_pw > {vol_rlx_output_file.name}"
!submit -n 1 -w '01:00:00' -e QE_DISABLE_GGA_PBE=0 --runName vcrlx {COMMANDvcrlx} {pseudos} -i {vol_rlx_input_file.name}
vol_rlx_output_file.close()

### Extract QE outputs using pymatgen

In [15]:
from pymatgen.io.pwscf import PWOutput

# Extract outputs
output_set = PWOutput('vcrlx.stdout')

In [16]:
output_set.final_energy

-75.45258459

In [17]:
# Extract stress manually
output = open('vcrlx.stdout', "r")
lines = output.readlines()
P = []
job_time = []
for line in lines:
    if 'P=' in line:
        data = line.split()
        P.append(float(data[5]))
print(P)

[872.4, 868.72]
